In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('C:/Users/oxana/Documents/Test_task/sleep.csv')

In [3]:
df.head()

,Unnamed: 0,indicator,time_offset
0,2017-12-01 21:00:00,0,-60
1,2017-12-01 21:01:00,0,-59
2,2017-12-01 21:02:00,0,-58
3,2017-12-01 21:03:00,0,-57
4,2017-12-01 21:04:00,0,-56


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11340 entries, 0 to 11339
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Unnamed: 0   11340 non-null  object
 1   indicator    11340 non-null  int64 
 2   time_offset  11340 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 265.9+ KB


In [5]:
df.columns = ['date', 'indicator', 'time_offset']

In [6]:
df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d %H:%M:%S')

In [7]:
df['date'].describe()

count                   11340
unique                  11340
top       2018-01-28 23:31:00
freq                        1
first     2017-12-01 21:00:00
last      2018-02-01 23:59:00
Name: date, dtype: object

In [8]:
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day

In [9]:
df['month'].unique()

array([12,  1,  2], dtype=int64)

In [10]:
df['time'] = df['date'].dt.strftime('%H:%M:%S')
df['time'] 

0        21:00:00
1        21:01:00
2        21:02:00
3        21:03:00
4        21:04:00
           ...   
11335    23:55:00
11336    23:56:00
11337    23:57:00
11338    23:58:00
11339    23:59:00
Name: time, Length: 11340, dtype: object

In [11]:
sleep_time = df.query('indicator == 1').groupby(['month', 'day'])['date'].min().reset_index()
sleep_time.head()

,month,day,date
0,1,1,2018-01-01 21:39:00
1,1,2,2018-01-02 22:21:00
2,1,3,2018-01-03 22:43:00
3,1,4,2018-01-04 21:55:00
4,1,5,2018-01-05 22:11:00


In [12]:
sleep_time['time'] = sleep_time['date'].dt.strftime('%H:%M:%S')
sleep_time.head()

,month,day,date,time
0,1,1,2018-01-01 21:39:00,21:39:00
1,1,2,2018-01-02 22:21:00,22:21:00
2,1,3,2018-01-03 22:43:00,22:43:00
3,1,4,2018-01-04 21:55:00,21:55:00
4,1,5,2018-01-05 22:11:00,22:11:00


In [13]:
sleep_time.head()

,month,day,date,time
0,1,1,2018-01-01 21:39:00,21:39:00
1,1,2,2018-01-02 22:21:00,22:21:00
2,1,3,2018-01-03 22:43:00,22:43:00
3,1,4,2018-01-04 21:55:00,21:55:00
4,1,5,2018-01-05 22:11:00,22:11:00


In [14]:
sleep_time['time'].min()

'21:07:00'

In [15]:
sleep_time['time'].max()

'23:27:00'

Пока можно сказать одно - в 23:27 человек уже всегда спит, а до 21:07 еще бодрствует.   
Но я забыла про поправку на время. ⏳ 😱    
Допустим, что нам даны секунды в столбце `time_offset`

In [16]:
df['time'] = pd.to_timedelta(df['time'])
df['seconds'] = df['time'].dt.total_seconds()

In [17]:
df['final_time'] = df['seconds'] + df['time_offset']

In [18]:
df['final_time'] = pd.to_datetime(df['final_time'], unit='s').dt.strftime("%H:%M:%S")

In [19]:
sleep = df.query('indicator == 1 and final_time > "20:00:00"').groupby(['month', 'day'])['final_time'].min().reset_index()
sleep.head()

,month,day,final_time
0,1,1,21:38:39
1,1,2,22:21:21
2,1,3,22:43:43
3,1,4,21:54:55
4,1,5,22:11:11


In [20]:
sleep['final_time'].min()

'21:06:07'

In [21]:
df.query('indicator == 1')['final_time'].min()

'00:00:59'

С учетом поправок на время (мы предположили, что поправки были в секундах) получилось, что самое раннее время, когда человек спал - 21:06:07, а самое позднее - 00:00:59.   
Т.е. в одну минуту после полуночи человек спит со 100% вероятностью

На остальные вопросы у меня пока нет ответов
### *I did my best* 🙂